In [2]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM  

In [3]:
tokenizer = AutoTokenizer.from_pretrained("llmware/bling-1.4b-0.1")  


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
model = AutoModelForCausalLM.from_pretrained("llmware/bling-1.4b-0.1")  



In [34]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

bs4_strainer = bs4.SoupStrainer(class_=("main-content"))
loader = WebBaseLoader(
    web_paths=("https://deltek.com/en","https://www.deltek.com/en/about/contact-us", "https://www.deltek.com/en/small-business"),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()



text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
all_splits_text = [split.page_content for split in all_splits]



In [84]:
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from sentence_transformers import SentenceTransformer
from langchain.embeddings import OpenAIEmbeddings


doc_embeddings = SentenceTransformer('all-MiniLM-L6-v2').encode(all_splits_text)
query = "Give me 3 different reasons Why is deltek liked by customers?"
query_embeddings = SentenceTransformer('all-MiniLM-L6-v2').encode(query)
print(doc_embeddings.shape)
pairs = zip(all_splits_text, doc_embeddings)

vectorstore = FAISS.from_embeddings(pairs, SentenceTransformer('all-MiniLM-L6-v2'))



(27, 384)


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [92]:
from sklearn.metrics.pairwise import cosine_similarity

results = cosine_similarity(doc_embeddings, query_embeddings.reshape(1,-1)).reshape((-1,))
k = 4
ixs = results.argsort()
ixs = ixs[::-1]

relevant_docs = []
results2 = []
for i in ixs:
    relevant_docs.append(all_splits[i].page_content)
    results2.append(results[i])
relevant_docs = relevant_docs[:k]
formatted_docs = "\n\n".join(doc for doc in relevant_docs)


In [87]:
context = formatted_docs

In [88]:
entries = {"context": context, 
          "query":query}


In [89]:
new_prompt = "<human>: " + entries["context"] + "\n" + entries["query"] + "\n" + "<bot>:"

inputs = tokenizer(new_prompt, return_tensors="pt")  
start_of_output = len(inputs.input_ids[0])

In [90]:
outputs = model.generate(
        inputs.input_ids.to("cpu"),
        attention_mask=inputs.attention_mask.to("cpu"),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.3,
        max_new_tokens=100,
        )


In [91]:
output_text = tokenizer.decode(outputs[0][start_of_output:], skip_special_tokens=True)
print(output_text)

 •Customer satisfaction: Deltek's solutions are the preferred choice for small businesses to help fuel business growth.
•Customer satisfaction: Deltek's solutions are the preferred choice for small businesses to help fuel business growth.
•Customer satisfaction: Deltek's solutions are the preferred choice for small businesses to help fuel business growth.
